In [1]:
import torch
import torchvision.models as models

from idsds.models.resnet import resnet18, resnet50, resnet101, resnet152, wide_resnet50_2
from idsds.models.vgg import vgg16, vgg16_bn, vgg13, vgg19, vgg11
from idsds.models.ViT.ViT_new import vit_base_patch16_224
from idsds.models.ViT.ViT_LRP import vit_base_patch16_224 as vit_LRP
from idsds.models.bagnets.pytorchnet import bagnet33
from idsds.models.xdnns.xfixup_resnet import xfixup_resnet50, fixup_resnet50
from idsds.models.xdnns.xvgg import xvgg16
from idsds.models.bcos_v2.bcos_resnet import resnet50 as bcos_resnet50
from idsds.models.bcos_v2.bcos_resnet import resnet18 as bcos_resnet18

import utils

original_models = "/workspace/hd/original/"
tuned_models = "/workspace/hd/tuned/"

In [6]:
import torch.nn.functional as F
import matplotlib.pyplot as plt

def compare_mad_attr(model1, model2, layer1, layer2, attr_fn, loader):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model1.to(device).eval()
    model2.to(device).eval()

    total_mad = 0
    num_samples = 0

    cam1 = attr_fn(model1, target_layer=layer1)
    cam2 = attr_fn(model2, target_layer=layer2)

    for images, _ in loader:
        images = images.to(device)

        for i in range(images.size(0)):
            input_tensor = images[i].unsqueeze(0)
    
            output1 = model1(input_tensor)
            pred_class1 = output1.argmax(dim=1)[0].item()
            attribution_map1 = cam1(pred_class1, output1)[0]
        
            output2 = model2(input_tensor)
            pred_class2 = output2.argmax(dim=1)[0].item()
            attribution_map2 = cam2(pred_class2, output2)[0]
        
            if attribution_map1.ndim == 3 and attribution_map1.shape[0] == 1:
                attribution_map1 = attribution_map1.squeeze(0)
            if attribution_map2.ndim == 3 and attribution_map2.shape[0] == 1:
                attribution_map2 = attribution_map2.squeeze(0)
            
            map1_resized = F.interpolate(attribution_map1.unsqueeze(0).unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze()
            map2_resized = F.interpolate(attribution_map2.unsqueeze(0).unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze()
        
            total_mad += torch.abs(map1_resized - map2_resized).mean().item()
        num_samples += images.size(0)
        print(f"{num_samples} {total_mad}")

    mad = total_mad / num_samples
    print(f"Mean Absolute Difference between attribution maps: {mad}")

test_loader = utils.get_loader("/workspace/hd/imagenet-mini/val", batch_size=500)

In [5]:
from torchcam.methods import GradCAM

resnet50_ood = resnet50(pretrained=True)
resnet50_id = resnet50(pretrained=True)
resnet50_id = utils.load_state_dict(
    tuned_models + "resnet50_imagenet1000_lr0.001_epochs30_step10_checkpoint_best.pth.tar", 
    resnet50_id
)
compare_mad_attr(resnet50_ood, resnet50_id, GradCAM, test_loader)
# 0.053905330118211615

model loaded
model loaded


/workspace/src/experiments/network-similarity/utils.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)


500 22.87311269622296
1000 44.498108664061874
1500 68.4017807324417
2000 95.65968740312383
2500 125.03875130461529
3000 154.36896573146805
3500 186.20815494144335
3923 211.47061005374417
Mean Absolute Difference between attribution maps: 0.053905330118211615


In [8]:
vgg16_ood = vgg16(pretrained=True)
vgg16_id = vgg16(pretrained=True)
vgg16_id = utils.load_state_dict(
    tuned_models + "/vgg16_imagenet1000_lr0.001_epochs30_step10_checkpoint_best.pth.tar", 
    vgg16_id
)
compare_mad_attr(vgg16_ood, vgg16_id, "features", "features", GradCAM, test_loader)
# 0.04445504777481494

500 19.695862133055925
1000 39.15862245950848
1500 59.35514400340617
2000 82.85035688849166
2500 104.98470893176273
3000 131.4548487327993
3500 156.28242710512131
3923 174.397152420599
Mean Absolute Difference between attribution maps: 0.04445504777481494
